Try to merge in overloads from handcoded and other maintainers 
----

- clone to repos 
- new docstubs 
- merge in overloads

# Merge with the notebook on another branch where I have already seperated out _mpy_shed 
- path repos\micropython-stubs\stubs\_manual\_mpy_shed
- but now also changes in this file 


#### todo 
enrich  
- [ ] --no-docstring still copies over the class docstring in array.array
- [x] check if types for module constants are copied over 
   - [x] Create method to Copy TypeVars and TypeAliasses from source to target ( needed for future merges) 
        - [x] TypeVar
        - [x] TypeAlias
        - cmath has some incorrect type definitions for _C 
- [ ] Bug: Imports are copied even if they are inside an if statement (repos\rp2040\rp2\asm_pio.pyi :`from rp2 import PIOASMEmit` ) 
- [ ] ? create a method to forcefully copy over a class and all methods while merging 


- [x] avoid defining AnyReadableBuf and AnyWritableBuf in multile modules ( _mpy_shed module)
- [x] for now: Include in docstub MODULE_GLUE 
- [x] manual addition of some classes
- [ ] rp2 and _rp2 modules need manual adjustments
    - [ ] there are quite a few classes that are not/partly documented .rst files
    - [ ] the several classes in `_rp2` are documented to be in `rp2` module
        - [ ] `PIOASMEmit` class is missing in `rp2` module
        - may also need a way to remove a class from a module



 copy from C:\Users\josverl\.vscode\extensions\ms-python.vscode-pylance-2024.9.1\dist\typeshed-fallback\stdlib\cmath.pyi


In [1]:
# set parameters

from pathlib import Path



version = "v1.24.0"


flat_version = version.replace(".", "_")


reference_p = Path("./scratch/micropython-reference")

MIT = False
if MIT:
    # Howards Typeshed
    overload_typestubs = Path("repos\\PyBoardTypeshed\\micropython_typesheds")
else:
    # IntelliJ Micropython plugin
    overload_typestubs = Path("repos\\intellij-micropython\\typehints")

In [2]:
# BLANK SLATE
! git -C scratch reset .
! git -C scratch clean . -f

# remove the old ref folder if it exists
if reference_p.exists():
    # remove the folder if it exists
    ! rmdir /s /q {reference_p}

# create docstubs without cleaning up the .rst  docstrings
! stubber docstubs --version {version} --stub-path scratch --no-autoflake 
# --no-clean-rst

! ren .\scratch\micropython-{flat_version}-docstubs micropython-reference

! black scratch
! git -C scratch add .

Unstaged changes after reset:
M	micropython-reference/_mpy_shed/__init__.pyi
M	micropython-reference/_thread/__init__.pyi
M	micropython-reference/array/__init__.pyi
M	micropython-reference/asyncio/__init__.pyi
M	micropython-reference/binascii/__init__.pyi
M	micropython-reference/bluetooth/__init__.pyi
M	micropython-reference/btree/__init__.pyi
M	micropython-reference/cmath/__init__.pyi
M	micropython-reference/collections/__init__.pyi
M	micropython-reference/cryptolib/__init__.pyi
M	micropython-reference/deflate/__init__.pyi
M	micropython-reference/errno/__init__.pyi
M	micropython-reference/esp/__init__.pyi
M	micropython-reference/esp32/__init__.pyi
M	micropython-reference/espnow/__init__.pyi
M	micropython-reference/framebuf/__init__.pyi
M	micropython-reference/gc/__init__.pyi
M	micropython-reference/gzip/__init__.pyi
M	micropython-reference/hashlib/__init__.pyi
M	micropython-reference/heapq/__init__.pyi
M	micropython-reference/io/__init__.pyi
M	micropython-reference/json/__init__.pyi
M

Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\_thread\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\collections\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\deflate\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\array\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\errno\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\esp\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\binascii\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\framebuf\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\esp32\__init__.pyi
reformatted D:\mypython

In [3]:
! git -C scratch reset .
# assume root of workspace is the current directory

# pull in josverl's handcoded rp2040 stubs
!stubber enrich --params-only --stubs scratch --docstubs repos\rp2040


Unstaged changes after reset:
D	micropython-reference/_mpy_shed/__init__.pyi
D	micropython-reference/_rp2/DMA.pyi
D	micropython-reference/_rp2/Flash.pyi
D	micropython-reference/_rp2/PIO.pyi
D	micropython-reference/_rp2/StateMachine.pyi
D	micropython-reference/_rp2/__init__.pyi
D	micropython-reference/_rp2/irq.pyi
M	micropython-reference/_thread/__init__.pyi
M	micropython-reference/array/__init__.pyi
M	micropython-reference/asyncio/__init__.pyi
M	micropython-reference/binascii/__init__.pyi
M	micropython-reference/bluetooth/__init__.pyi
M	micropython-reference/btree/__init__.pyi
M	micropython-reference/cmath/__init__.pyi
M	micropython-reference/collections/__init__.pyi
M	micropython-reference/cryptolib/__init__.pyi
M	micropython-reference/deflate/__init__.pyi
M	micropython-reference/errno/__init__.pyi
M	micropython-reference/esp/__init__.pyi
M	micropython-reference/esp32/__init__.pyi
M	micropython-reference/espnow/__init__.pyi
M	micropython-reference/framebuf/__init__.pyi
M	micropython-r

                                                                               
 Usage: stubber enrich [OPTIONS]                                               
                                                                               
 Try 'stubber enrich --help' for help                                          
┌─ Error ─────────────────────────────────────────────────────────────────────┐
│ Invalid value for '--docstubs' / '--source' / '-ds': Path 'repos\\rp2040'   │
│ does not exist.                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
                                                                               


In [4]:
### sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [5]:
## Merge redacted typehints from the overload source with docstubs
!stubber enrich --params-only --stubs scratch --docstubs {overload_typestubs}


15:21:31 | ℹ️ micropython-stubber 1.23.2a0
15:21:31 | ℹ️ Enriching scratch with repos\intellij-micropython\typehints
15:21:31 | ℹ️ Enrich folder scratch.
15:21:31 | ℹ️ Merge scratch\micropython-reference\_thread\__init__.pyi from 
repos\intellij-micropython\typehints\stdlib\_thread.pyi
15:21:31 | ℹ️ Merge scratch\micropython-reference\array\__init__.pyi from 
repos\intellij-micropython\typehints\stdlib\array.pyi
15:21:31 | ℹ️ Change __getitem__ to @overload
15:21:31 | ℹ️ Change __setitem__ to @overload
15:21:31 | ℹ️ Merge scratch\micropython-reference\array\__init__.pyi from 
repos\intellij-micropython\typehints\stdlib\uarray.pyi
15:21:32 | ℹ️ Merge scratch\micropython-reference\asyncio\__init__.pyi from 
repos\intellij-micropython\typehints\stdlib\uasyncio.pyi
15:21:32 | ℹ️ Merge scratch\micropython-reference\binascii\__init__.pyi from 
repos\intellij-micropython\typehints\stdlib\binascii.pyi
15:21:32 | ℹ️ Merge scratch\micropython-reference\binascii\__init__.pyi from 
repos\intellij-

In [6]:
# sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [7]:
# add _rp2 module
! mkdir scratch\micropython-reference\_rp2
!copy .\repos\rp2040\_rp2\*.* .\scratch\micropython-reference\_rp2\*.*

The system cannot find the path specified.


In [8]:
import shutil


manual_p = Path("repos\\micropython-stubs\\stubs\\_manual")
# rp2040_p = Path("repos\\rp2040")


# copy _mpy_shed to the reference folder
def copy_module(module: str | Path, src: Path = manual_p):
    target = reference_p / module
    if target.exists():
        shutil.rmtree(target)
    if (src / module).is_dir():
        shutil.copytree(src / module, target)
    else:
        shutil.copy(src / module, target)

In [9]:
## Copy _mpy_shed to the reference folder
copy_module("_mpy_shed", manual_p)
copy_module("_thread", manual_p)

In [10]:
# Add rp2 modules ( may be an overkill?)
# copy_module("rp2", manual_p/ "rpi")
copy_module("_rp2", manual_p / "rpi")

In [11]:
# fix u-module imports and replace stdlib imports with _mpy_shed

from pathlib import Path


UMOD_TO_UMOD = [
    "from uarray import",
    "from uio import",
    "from usocket import",
    "from uos import",
    "from usys import",
    "from utime import",
]


NO_IMPORT = [
    "from io import AnyReadableBuf",
    "from io import AnyWritableBuf",
    "from uio import AnyReadableBuf",
    "from uio import AnyWritableBuf",
]


REPLACE = [
    ("from os import AbstractBlockDev", "from _mpy_shed import AbstractBlockDev"),
    ("from io import IOBase", "from _mpy_shed import IOBase"),
    ("from uio import IOBase", "from _mpy_shed import IOBase"),
]


# Iterate over all files in the directory
# reference_p = Path("scratch/micropython-v1_23_0-docstubs")


for file in reference_p.rglob("*.p*"):
    # Check if the file is a regular file
    if file.is_file():
        # Open the file in read mode
        with file.open(mode="r", encoding="utf-8") as f:
            # Read the contents of the file
            lines = f.readlines()

        # Open the file in write mode
        with file.open(mode="w", encoding="utf-8") as f:
            # Iterate over each line in the file
            for line in lines:
                if "import" not in line:
                    f.write(line)
                    continue

                # Check if the line starts with
                if any(line.startswith(l) for l in UMOD_TO_UMOD):
                    # Modify the line to remove the `u` prefix
                    line = "from " + line[6:]

                # always replace the imports
                for old, new in REPLACE:
                    if line.startswith(old):
                        line = f"{new}\n"
                        break

                if any(line.startswith(l) for l in NO_IMPORT):
                    # remove the line
                    # line = "# " + line
                    line = ""

                # Write the modified line to the file
                f.write(line)

In [12]:
# sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [13]:
# Add missing HASH class to hashlib

HASH_CLASS = open(manual_p / "partial_modules" / "hashlib.pyi").read()


with open(reference_p / "hashlib/__init__.pyi", "a") as f:

    f.write(HASH_CLASS)

In [14]:
# Add missing PIOASM Emit class to rp2

CLASS_PIOASMEMIT = open(manual_p / "partial_modules" / "rp2.pyi").read()


# Add missing class


rp2 = Path(reference_p / "rp2/__init__.pyi")


with open(rp2, "a") as f:


    f.write(CLASS_PIOASMEMIT)

In [15]:
# clean / change rp2 / _rp2
# remove the submodules  to avoid duplication in multiple files
for name in ["DMA", "FLASH", "PIO", "StateMachine"]:
    submodule = reference_p / f"rp2/{name}.pyi"
    submodule.unlink(missing_ok=True)


# remove the relative imports from the rp2 module
with rp2.open(mode="r", encoding="utf-8") as f:
    # Read the contents of the file
    lines = f.readlines()

# Open the file in write mode
with rp2.open(mode="w", encoding="utf-8") as f:
    # Iterate over each line in the file
    for line in lines:
        # Check if the line starts with
        if line.startswith("from .") or line == "from rp2 import PIOASMEmit":
            line = "# " + line

        # Write the modified line to the file
        f.write(line)

In [16]:
# Clean formatting and upgrate typing notation to 3.10
! ruff check {reference_p} --fix --unsafe-fixes


Found 1064 errors (1064 fixed, 0 remaining).


In [17]:
# run autoflake on all files - one by one to get the __init__.pyi files
for f in reference_p.rglob("*.pyi"):
    ! autoflake {f} --in --imports typing_extensions,_mpy_shed,_typeshed


In [18]:
# clean up the formatting
! isort {reference_p}

Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\uselect.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\usocket.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\array\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\asyncio\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\bluetooth\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\btree\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\cmath\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\collections\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\cryptolib\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\deflate\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\errno\__init__.pyi
Fixing D:\mypython\micropyth

In [19]:

!cd "d:\mypython\micropython-stubber\scratch\micropython-reference" && pyright .
!cd "d:\mypython\micropython-stubber"


Please install the new version or set PYRIGHT_PYTHON_FORCE_VERSION to `latest`

d:\mypython\micropython-stubber\scratch\micropython-reference\esp32\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-reference\esp32\__init__.pyi:68:14 - information: Class methods should take a "cls" parameter (reportSelfClsParameterName)
  d:\mypython\micropython-stubber\scratch\micropython-reference\esp32\__init__.pyi:121:40 - information: Class methods should take a "cls" parameter (reportSelfClsParameterName)
  d:\mypython\micropython-stubber\scratch\micropython-reference\esp32\__init__.pyi:157:21 - information: Class methods should take a "cls" parameter (reportSelfClsParameterName)
d:\mypython\micropython-stubber\scratch\micropython-reference\espnow\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-reference\espnow\__init__.pyi:468:35 - information: "value" is not defined (reportUndefinedVariable)
d:\mypython\micropython-stubber\scratch\micropython-reference\machine